In [12]:
import pandas as pd
import pandas_ml as pdml
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.decomposition import PCA


In [13]:
#Load data
df = pd.read_csv('data.csv');
#Replace NaN with 0
df = df.fillna(0);
df['DATE'] = pd.to_datetime(df['DATE']);
df.describe()

,Average Wind Speed,PRCP,TMAX,TMIN,Fastest 2 minute wind speed,Fastest 5 second wind speed,Fog/Ice,Heavy/Freezing Fog,Thunder,Smoke/Haze,Power Outage
count,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000
mean,7.274945,0.079595,59.249828,37.227835,18.562715,28.019416,0.344674,0.100344,0.053265,0.150172,0.054639
std,3.533600,0.225217,21.368136,18.345112,7.288373,17.480074,0.475343,0.300509,0.224599,0.357301,0.227314
min,0.000000,0.000000,0.000000,-30.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.700000,0.000000,41.000000,25.000000,14.100000,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.930000,0.000000,62.000000,38.000000,17.000000,23.900000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.400000,0.030000,78.000000,52.000000,21.900000,31.100000,1.000000,0.000000,0.000000,0.000000,0.000000
max,22.150000,4.000000,102.000000,73.000000,83.000000,185.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
#Add columns for wind squared
df['Avg Wind Speed Squared'] = pd.Series(np.square(df['Average Wind Speed']), index=df.index)
df['5 second wind speed squared'] = pd.Series(np.square(df['Fastest 5 second wind speed']));
df['2 min wind speed squared'] = pd.Series(np.square(df['Fastest 2 minute wind speed']));
df = df[['Power Outage', 'PRCP', 'TMAX','TMIN','Fastest 5 second wind speed','5 second wind speed squared', 'Fastest 2 minute wind speed' ,'2 min wind speed squared','Average Wind Speed','Avg Wind Speed Squared','Fog/Ice', 'Heavy/Freezing Fog', 'Thunder','Smoke/Haze']];


In [15]:
#Under sample based on Cluster Centroids
df_ML = pdml.ModelFrame(df, target = 'Power Outage')
df_ML.target_name
print(df_ML.target.value_counts());
sampler = df_ML.imbalance.under_sampling.ClusterCentroids();
sampled = df_ML.fit_sample(sampler);
print(sampled.target.value_counts());
finalData = pd.DataFrame(sampled);
finalData.describe()

0.0    2751
1.0     159
Name: Power Outage, dtype: int64
1.0    159
0.0    159
Name: Power Outage, dtype: int64


,Power Outage,PRCP,TMAX,TMIN,Fastest 5 second wind speed,5 second wind speed squared,Fastest 2 minute wind speed,2 min wind speed squared,Average Wind Speed,Avg Wind Speed Squared,Fog/Ice,Heavy/Freezing Fog,Thunder,Smoke/Haze
count,318.000000,318.000000,318.000000,318.000000,318.000000,318.000000,318.000000,318.000000,318.000000,318.000000,318.000000,3.180000e+02,3.180000e+02,3.180000e+02
mean,0.500000,0.142731,63.187734,41.926335,50.132510,3845.285369,25.570283,804.617976,8.173478,86.405378,0.511540,1.503688e-01,1.317914e-01,2.098480e-01
std,0.500788,0.282982,20.967756,16.785533,36.552708,6327.579356,12.276899,930.241656,4.277876,91.182525,0.442045,3.122838e-01,3.110567e-01,3.568476e-01
min,0.000000,0.000000,0.000000,-8.000000,9.780556,99.140278,7.927778,65.253889,1.120000,1.254400,0.000000,-2.775558e-17,-2.775558e-17,-2.775558e-17
25%,0.000000,0.000000,46.000000,30.416667,25.392898,644.947788,16.876786,285.393036,5.140000,27.817217,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.500000,0.022158,67.677083,43.887179,36.853846,1358.900769,23.000000,529.000000,7.380000,54.464400,0.419872,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.000000,0.138697,79.500000,55.750000,62.000000,3844.000000,30.881250,954.181875,10.290000,109.039996,1.000000,1.073059e-01,2.895067e-02,2.000000e-01
max,1.000000,1.900000,102.000000,73.000000,185.000000,34225.000000,83.000000,6889.000000,22.150000,490.622500,1.000000,1.000000e+00,1.000000e+00,1.000000e+00


In [16]:
# Random Split
#Split the data in two parts.
# powerOutages = df.loc[df['Power Outage'] == 1];
# noPowerOutages = df.loc[df['Power Outage'] == 0];

# zeroDays = noPowerOutages.sample(159);
# finalData = pd.concat([powerOutages,zeroDays]);
# finalData = finalData.sample(318);


In [17]:
#Normalize data
scaledData = preprocessing.MinMaxScaler().fit_transform(finalData.values);
finalData = pd.DataFrame(scaledData, index = finalData.index, columns = finalData.columns);

In [18]:
finalData.to_csv('Clustered Data', index = False);